# Novojob.com

In [2]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrap_novojob():
    # Liste des liens pour chaque catégorie
    categories = [
        "toutes les offres d'emploi",
        "juridique,fiscal,audit,conseil",
        "administrations,moyens généraux",
        "assistanat,secrétariat",
        "metiers banque et assurances",
        "RH,personnel,formation",
        "education,enseignement",
        "direction générale,direction d'unité",
        "vente,televente,assistanat",
        "commercial,technico commercial,service client",
        "responsable commercial,grands comptes",
        "créatio, design",
        "marketing, communication",
        "journalisme,médias,traduction",
        "informatique,systèmes d'information,internet",
        "télécommunication,réseaux",
        "chantier,métiers BTP,architecture",
        "ingénierie,etudes,projet,R&D",
        "logistique,achat,stock,transport",
        "production,méthode,industrie",
        "maintenance,entretien",
        "Qualité,sécurité,Environnement",
        "Santé,Médical,Pharmacie",
        "Hotellerie,Tourisme,Restauration, Loisirs",
        "Ouvriers qualifiés, Chauffeur",
        "autre",
        "Métiers de l'agriculture"
    ]

    base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
    category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    intitules_list = []
    entreprises_list = []
    pays_list = []
    dates_list = []
    niveau_list = []
    experience_list = []

    url_list = []
    all_job_lien = []

    # Parcourir les liens de chaque catégorie
    for category_link in category_links:
        req = requests.get(category_link, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='row-fluid job-details pointer')

        for offre in offres:
            offre_link_tag = offre.find('a')
            if offre_link_tag:
                offre_link = offre_link_tag['href']
                all_job_lien.append(offre_link)

            entreprise_tag = offre.find('h6', class_='ellipsis')
            entreprise = entreprise_tag.get_text().strip() if entreprise_tag else None

            intitule_tag = offre.find('h2', class_='ellipsis row-fluid')
            intitule = intitule_tag.get_text().strip() if intitule_tag else None

            bloc_bottom = offre.find_next('div', class_='bloc-bottom')

            pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
            pays = pays_info.find_parent().text.strip() if pays_info else None

            date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
            date = date_info.find_parent().text.strip() if date_info else None

            niveau_info = bloc_bottom.find('i', class_='fa fa-bookmark icon-left')
            niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

            match = re.match(r'(.+) \((.+)\)', niveau_text)
            niveau_col, experience_col = match.groups() if match else (None, None)

            intitules_list.append(intitule)
            entreprises_list.append(entreprise)
            pays_list.append(pays)
            dates_list.append(date)
            niveau_list.append(niveau_col)
            experience_list.append(experience_col)
            
            url_list.append(category_link)

    # Création du DataFrame
    df_offers = pd.DataFrame({
        'Intitule': intitules_list,
        'Entreprise': entreprises_list,
        'Pays': pays_list,
        'Date': dates_list,
        'Niveau': niveau_list,
        'Experience_lettre': experience_list,
        'url': url_list,
        'Offre_Link': all_job_lien
    })

    # Importer les détails de chaque offre d'emploi
    job_urls = list(df_offers['Offre_Link'])
    all_job_details = []

    for url in job_urls:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        job_details = {}
        
        # Ajouter le lien
        job_details["Offre_Link"] = url
        
        # Extraction des détails de l'offre d'emploi
        details_section = soup.find('ul', class_='text-small')
        if details_section:
            for li in details_section.find_all('li', class_='row-fluid'):
                key = li.find('span', class_='span4').text.strip()
                value = li.find('span', class_='span8').text.strip()
                job_details[key] = value

        # Extraction du texte fourni
        description_section = soup.find('div', class_='spaced details-description')
        if description_section:
            provided_text = description_section.text.strip()
            job_details['Provided Text'] = provided_text

        all_job_details.append(job_details)

    # Création d'un DataFrame pour les détails des offres d'emploi
    df_Novojob = pd.DataFrame(all_job_details)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Novojob['Intitule'] = intitules_list
    df_Novojob['Entreprise'] = entreprises_list
    df_Novojob['Pays'] = pays_list
    df_Novojob['Date'] = dates_list
    df_Novojob['Niveau'] = niveau_list
    df_Novojob['Experience_lettre'] = experience_list
    df_Novojob['url_Lien'] = url_list
    df_Novojob['Offre_Link1'] = all_job_lien
    
    equivalences = {
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Nom de l'entreprise": "RAISON_SOCIALE_DE_L_ENTREPRISE",
    "Secteur d'activité": "BRANCHE_D_ACTIVITE",
    "Lieu de travail": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Date d'expiration": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Nombre de postes": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Niveau de poste": None,
    "Niveau d'étude (diplôme)": "NIVEAU_D_ETUDES",
    "Type de contrat": "TYPE_DE_CONTRAT_DU_POSTE",
    "Provided Text": None,
    "Intitule": "INTITULE_DU_POSTE",
    "Entreprise": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL",
    "Date": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Niveau": None,
    "Experience_lettre": None,
    "url_Lien": "Offre_Link1"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    df_Novojob = renommer_colonnes(df_Novojob, equivalences)

    # Réorganiser les colonnes selon vos besoins

    return df_Novojob

# Appel de la fonction principale
scrap_novojob()

,SITE_WEB_DE_L_ENTREPRISE,RAISON_SOCIALE_DE_L_ENTREPRISE,BRANCHE_D_ACTIVITE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,NOMBRE_DE_POSTES_A_POURVOIR,Niveau de poste,Niveau d'étude (diplome),TYPE_DE_CONTRAT_DU_POSTE,Provided Text,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,Offre_Link1,Offre_Link1
0,https://www.novojob.com/cote-d-ivoire/offres-d...,Entreprise anonyme,Services,Côte d'ivoire,03 Avril,40 postes ouverts,Débutant / Junior| Stagiaire / Etudiant| Confi...,Niveau secondaire| Niveau terminal| Baccalauré...,Mission,SPPCI Offre d'emploiLivreurs Moto (H/F)Vous êt...,Livreurs Moto,Entreprise anonyme,Côte d'ivoire,04 Janvier,Confirmé / Expérimenté,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
1,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",19 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...,Pompiste H/F,None,"Abidjan, Côte d'ivoire",27 Décembre 2023,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
2,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,Côte d'ivoire,26 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...,Caissier H/F,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
3,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,Côte d'ivoire,26 Mars,01,Débutant / Junior,NaN,NaN,Coris consulting ci est un cabinet d'insertion...,Ouvrier de rayon,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
4,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Informatique, Télécom, Internet",Côte d'ivoire,02 Mai,25,Débutant / Junior| Stagiaire / Etudiant| Jeune...,"TS Bac +2| Licence (LMD), Bac + 3",Consultant,KOMIAN IA est une entreprise de technologie d’...,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Banque, Assurance, Finance","Abidjan, Côte d'ivoire",05 Mai,01,Confirmé / Expérimenté,"Master 1, Licence Bac + 4| Master 2, Ingénior...",CDI,"Présentation de l’Entreprise La SIB, Société I...",Responsable Clientèle Patrimoniale (H/F),Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",05 Février,Confirmé / Expérimenté,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
704,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"BTP, Construction, Immobilier",Côte d'ivoire,16 Avril,01,Débutant / Junior,"Master 2, Ingéniorat, Bac + 5",CDD,SOCOPI recrute un technicien génie civil optio...,Technicien Génie Civil Option Bâtiment,SOCOPI,Côte d'ivoire,17 Janvier,Débutant / Junior,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
705,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Distribution, Commerce",Côte d'ivoire,26 Mai,01,Débutant / Junior| Confirmé / Expérimenté,NaN,NaN,Vous êtes passionné(e) par le développement d'...,Développeur iOS et web H/F,MONCONFORT,Côte d'ivoire,26 Février,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...
706,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Côte d'ivoire| Adzope, Côte d'ivoire| Agbovill...",09 Mars,01,Débutant / Junior| Confirmé / Expérimenté,NaN,NaN,Description de l’organisationmyAgro est u

# Emploi.educarriere.ci

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import urllib3

def emploi_educarriere():
    # Ignorer les avertissements SSL
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    # Fonction pour extraire le texte d'un élément HTML
    def extract_text(element, class_name=None, style=None, text_contains=None):
        if element:
            tag = element.find(class_=class_name, style=style, text=text_contains)
            return tag.text.strip() if tag else ""
        else:
            return ""

    # Fonction pour nettoyer le texte
    def clean_text(text):
        if text is not None:
            cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
            return cleaned_text.strip() if cleaned_text else None
        else:
            return None

    # Fonction pour extraire la date à partir d'un élément HTML
    def extract_date(element, text_contains):
        date_elements = element.find_all('a', class_='text')
        date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
        return date

    # Fonction pour extraire les détails des offres d'emploi
    def scrap_job_details(urls):
        # En-tête pour éviter d'être bloqué
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
        options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
        chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
        options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
        options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
        driver = webdriver.Chrome(options=options)

        # Liste pour stocker les détails de chaque emploi
        all_job_details = []

        # Parcourir les liens
        for url in urls:
            req = requests.get(url, headers=headers)
            soup = BeautifulSoup(req.text, 'html.parser')
            time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

            offres = soup.find_all('div', class_='box row')

            # Parcourir les offres d'emploi sur la page principale
            for offre in offres:
                # Trouver la balise <h4> dans la structure HTML pour extraire le lien
                offre_link_tag = offre.find('h4')

                # Vérifier si la balise <h4> a été trouvée
                if offre_link_tag:
                    # Extraire le lien de l'attribut 'href'
                    offre_link = offre_link_tag.find('a')['href']
                    all_job_details.append({'Offre_Link': offre_link})

        # Fermer le pilote Selenium à la fin
        driver.quit()

        # Concaténer tous les détails des emplois en un seul DataFrame
        if all_job_details:
            all_job_details_df = pd.DataFrame(all_job_details)
            return all_job_details_df
        else:
            print("Aucun détail d'offre d'emploi trouvé.")
            return None

    # Fonction pour récupérer les données des offres d'emploi
    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=200)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='box row')

        data_list = []

        for wrapper in job_description_wrappers:
            h4_tag = wrapper.find('h4')
            poste = extract_text(h4_tag)

            entry_title_tag = wrapper.find('p', class_='entry-title')
            sous_titre = extract_text(entry_title_tag)

            a_text_tag = wrapper.find('a', class_='text')
            code = extract_text(a_text_tag, style='color:#FF0000;')

            date_edition = extract_date(wrapper, "Date d'édition:")
            date_limite = extract_date(wrapper, "Date limite:")

            pays_tag = wrapper.find('a', class_='text')
            pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

            sous_titre = clean_text(sous_titre)

            data_list.append({
                'Poste': clean_text(poste),
                'Sous_titre': sous_titre,
                'Code': clean_text(code),
                'Date_DEdition': date_edition,
                'Date_limite': date_limite,
                'Pays': clean_text(pays),
                'URL': url  # Ajout de la colonne 'URL'
            })

        df = pd.DataFrame(data_list)
        return df

    # Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
    def add_offre_link_column(result_df):
        job_details_df = scrap_job_details(list(result_df["URL"]))
        if job_details_df is not None:
            # Join des DataFrames
            result_df = pd.concat([result_df, job_details_df], axis=1)
            return result_df
        else:
            print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
            return None

    # Fonction pour extraire les informations de l'offre d'emploi
    def extract_job_information(soup, url):
        try:
            # Extraction des informations de l'offre d'emploi
            poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
            type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
            metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
            niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
            experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
            lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
            
            # Extraction des dates de publication et de limite
            date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
            date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
            
            # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
            description = soup.select_one('div.entry-content').text.strip()

            return {
                "Poste": [poste],
                "Type d'offre": [type_offre],
                "Métier(s)": [metiers],
                "Niveau(x)": [niveaux],
                "Expérience": [experience],
                "Lieu": [lieu],
                "Offre_Link": [url],
                "Date de publication": [date_publication],
                "Date limite": [date_limite],
                "Description": [description]
            }
        except Exception as e:
            print(f"An error occurred while extracting job information for URL {url}: {e}")
            # Retourner un dictionnaire avec l'URL en cas d'erreur
            return {"Offre_Link": [url]}

    # Liste des liens
    urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

    # Créer un DataFrame à partir des liens
    result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

    # Ajouter la colonne 'Offre_Link'
    result_df = add_offre_link_column(result_df)
    # Supprimer les lignes avec plus de 80% de valeurs NaN
    threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
    result_df = result_df.dropna(thresh=threshold)
    result_df = result_df.reset_index(drop=True)
    
    # Liste des URLs à scraper
    urls = list(result_df['Offre_Link'])

    # Liste pour stocker les DataFrames
    dfs = []

    # Boucle sur chaque URL
    for url in urls:
        try:
            # Envoyer une requête GET au site avec un délai de 120 secondes
            response = requests.get(url, headers=headers, verify=True, timeout=120)

            # Vérifier si la requête a réussi (statut 200)
            if response.status_code == 200:
                # Analyser le contenu de la page avec BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup, url)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            else:
                print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
                # Ajouter une ligne avec l'URL en cas d'erreur
                dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        except requests.exceptions.Timeout:
            print(f"Timeout lors de la requête pour l'URL {url}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))
        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
        time.sleep(5)

    # Concaténer tous les DataFrames en un seul DataFrame
    df_Educarriere = pd.concat(dfs, ignore_index=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Educarriere['Poste'] = list(result_df['Poste'])
    df_Educarriere['Sous_titre'] = list(result_df['Sous_titre'])
    df_Educarriere['Code'] = list(result_df['Code'])
    df_Educarriere['Date_DEdition'] = list(result_df['Date_DEdition'])
    df_Educarriere['Date_limite'] = list(result_df['Date_limite'])
    df_Educarriere['Pays'] = list(result_df['Pays'])
    equivalences = {
    "Poste": "INTITULE_DU_POSTE",
    "Type d'offre": "TYPE_DE_CONTRAT_DU_POSTE",
    "Métier(s)": "SPECIALITE",
    "Niveau(x)": None,
    "Expérience": "EXPERIENCE_PROFESSIONNELLE",
    "Lieu": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Date de publication": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Date limite": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Description": None,
    "Sous_titre": None,
    "Code": None,
    "Date_DEdition": None,
    "Date_limite": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    
    # Renommer les colonnes du DataFrame
    df_Educarriere = renommer_colonnes(df_Educarriere, equivalences)

    # Réorganiser les colonnes selon vos besoins
    # Vous pouvez réorganiser les colonnes ici

    # Afficher ou retourner le DataFrame selon votre besoin
    return df_Educarriere

# Appel de la fonction principale

emploi_educarriere()

,INTITULE_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,SPECIALITE,Niveau(x),EXPERIENCE_PROFESSIONNELLE,LIEU_DU_POSTE_DE_TRAVAIL,SITE_WEB_DE_L_ENTREPRISE,DATE_DE_DEBUT_DE_L_OFFRE,DATE_D_EXPIRATION_DE_L_OFFRE,Description,Sous_titre,Code,Date_DEdition,Date_limite,PAYS_DU_POSTE_DE_TRAVAIL
0,OFFRES D'EMPLOI DANS LE DOMAINE DE LA SECURITE...,Emploi,Finances/Comptabilité,"BAC+4, BAC+5",6 ans,Abidjan,https://emploi.educarriere.ci/offre-114752-rev...,29/02/2024,11/03/2024,SECCrecruteREVISEUR COMPTABLE\n \nProfil du po...,Description du poste*** UNE IMPORTANTE SOCIETE...,110576,29/02/2024,24/03/2024,...
1,REVISEUR COMPTABLE,Emploi,"Banque, Commerce et Administration des Entrepr...",BAC+5,15 ans,Abidjan,https://emploi.educarriere.ci/offre-114751-dir...,29/02/2024,11/03/2024,CASHDEV AFRICArecruteDIRECTEUR COMMERCIAL AFRI...,SECCrecruteREVISEUR COMPTABLEProfil du posteLe...,110575,29/02/2024,11/03/2024,Abidjan
2,DIRECTEUR COMMERCIAL AFRIQUE,Stage,"Commerce et Administration des Entreprises, Co...","BAC+5, BAC+4, BAC+3, BAC+2, BAC+1, BAC, BT",,Abidjan Koumassi Immeuble Auchan,https://emploi.educarriere.ci/offre-114750-off...,29/02/2024,16/03/2024,Description du posteAprès la promotion de notr...,CASHDEV AFRICArecruteDIRECTEUR COMMERCIAL AFRI...,110574,29/02/2024,11/03/2024,Abidjan
3,OFFICE MANAGER,Emploi,Commerce/Ventes,BAC+2,2 ans,PLATEAU-DOKUI,https://emploi.educarriere.ci/offre-114749-com...,29/02/2024,15/03/2024,NUMERISrecruteCOMMERCIAL TERRAIN\n \n \nDescri...,DEEPER INSIGHTS CORPORATErecruteOFFICE MANAGER...,110573,29/02/2024,16/03/2024,K...
4,COMMERCIAL TERRAIN,Emploi,Juridique/Droit,BAC+5,3 ans,"ABIDJAN, COCODY FAYA, ROUTE DABATTA, Cité Pal...",https://emploi.educarriere.ci/offre-114747-jur...,29/02/2024,11/03/2024,FFIDUS CONSEILrecruteJURISTE FISCALISTE\n \n \...,NUMERISrecruteCOMMERCIAL TERRAINDescription du...,110572,29/02/2024,15/03/2024,PLATEAU-DOKUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,RESPONSABLE COMMERCIAL,Stage,Commerce/Ventes,"BAC+2, BAC+3, BAC+4, BAC+5",,Nos locaux sont situés sur lautoroute du nord...,https://emploi.educarriere.ci/offre-114704-sta...,28/02/2024,22/03/2024,BRASSIVOIRErecruteSTAGIAIRE CHANNEL\n \n \nDes...,AGENCE IVOIRE INTERIM (A2I)recruteRESPONSABLE ...,109191,25/01/2024,29/02/2024,ABIDJAN
506,DELEGUE MEDICAL,Emploi,Mécanique,BEPC,1 ans,Indéterminée,https://emploi.educarriere.ci/offre-114719-mec...,28/02/2024,11/03/2024,AKWA HOLDINGrecruteMÉCANICIEN\n \n \nDescripti...,Description du posteSuperviser par le responsa...,109176,25/01/2024,29/02/2024,R...
507,INGENIEUR DRIVE TEST,Emploi,"Juridique/Droit, Ressources Humaines , Science...","BAC+2, BAC+3",1 ans,Yopougon PK 24,https://emploi.educarriere.ci/offre-114718-sup...,28/02/2024,11/03/2024,EBURKA ConseilsrecruteSUPERVISEUR DE SITE RH\n...,Africa Project ManagementrecruteINGENIEUR DRIV...,109167,25/01/2024,30/03/2024,ABIDJAN
508,SUPERVISEUR HSE,Emploi,"Juridique/Droit, Ressources Humaines , Science...","BAC+2, BAC+3",1 ans,"Cocody, Angré 7e tranche, non loin du centre d...",https://emploi.educarriere.ci/offre-114717-ass...,28/02/2024,11/03/2024,EBURKA ConseilsrecruteAssistant RH chargé du T...,AGENCE IVOIRE INTERIM (A2I)recruteSUPERVISEUR ...,109146,24/01/2024,29/02/2024,ABIDJAN


# Alerteemploi.net soucis sur leur site

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def alerte_emploi():
    # Liste des URLs des offres d'emploi
    job_listing_urls = [
        "https://alerteemploi.net/toutes-les-offres/",
        # Ajoutez d'autres URLs au besoin
    ]

    # Scrapper les détails supplémentaires à partir des URL
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\Dell\\Desktop\\offre_d_emploi\\chromedriver-win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Configurez Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)

    # Liste pour stocker les données des offres d'emploi
    job_data = []

    # Boucle à travers chaque URL d'offre d'emploi
    for url in job_listing_urls:
        # Envoyez une requête GET en utilisant Selenium
        driver.get(url)

        while True:
            try:
                # Cliquez sur le bouton "Load more listings"
                load_more_button = driver.find_element(By.CLASS_NAME, 'load_more_jobs')
                load_more_button.click()

                # Attendez le chargement des nouvelles offres d'emploi
                WebDriverWait(driver, 10).until(
                    EC.invisibility_of_element_located((By.CLASS_NAME, 'loading_jobs'))
                )

                # Obtenez le code source de la page mis à jour
                page_source = driver.page_source

                # Utilisez BeautifulSoup pour analyser le HTML
                soup = BeautifulSoup(page_source, 'html.parser')

                # Trouvez toutes les offres d'emploi sur la page
                job_listings = soup.find_all('li', class_='job_listing')

                # Boucle à travers chaque offre d'emploi et extrayez les informations
                for listing in job_listings:
                    job_title = listing.find('div', class_='position').find('h3').text.strip()
                    company = listing.find('div', class_='company').find('strong').text.strip()
                    location = listing.find('div', class_='location').text.strip()
                    date_posted_element = listing.find('li', class_='date').find('time')
                    date_posted = date_posted_element['datetime'].strip() if date_posted_element else None
                    listing_url = listing.find('a')['href']

                    # Stockez les données dans un dictionnaire
                    job_entry = {
                        'Titre du poste': job_title,
                        'Entreprise': company,
                        'Lieu': location,
                        'Date de publication': date_posted,
                        'URL': listing_url,
                        'URL_offre': url
                    }

                    job_data.append(job_entry)

            except Exception as e:
                job_entry = {
                    'Titre du poste': "",
                    'Entreprise': "",
                    'Lieu': "",
                    'Date de publication': "",
                    'URL': "",
                    'URL_offre': url
                }
                # Sortez de la boucle si le bouton n'est pas trouvé ou s'il y a une exception
                break

    # Fermez le pilote Selenium
    driver.quit()

    # Créez un DataFrame
    df_alerteemploi = pd.DataFrame(job_data)

    # Liste d'URLs des pages d'offres d'emploi
    urls = list(df_alerteemploi["URL"])
    # Initialiser une liste pour stocker les données
    job_data = []

    # Configurer Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    # Boucle à travers chaque URL
    for url in urls:
        try:
            # Initialiser le pilote Selenium
            driver = webdriver.Chrome(options=chrome_options)

            # Envoyer une requête GET en utilisant Selenium
            driver.get(url)

            # Attendre quelques secondes (ajustez selon les besoins)
            time.sleep(30)

            # Attendre que la page soit entièrement chargée
            WebDriverWait(driver, 10).until(
                lambda x: x.execute_script("return document.readyState === 'complete'")
            )

            # Récupérer le code source de la page après l'exécution de JavaScript
            page_source = driver.page_source

            # Fermer le pilote Selenium
            driver.quit()

            # Utiliser BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extraire les détails de l'offre d'emploi
            job_title_element = soup.find('h1', class_='entry-title')
            job_title = job_title_element.text.strip() if job_title_element else None

            # Check if the initial element is found before attempting to find the nested element
            author_container = soup.find('div', class_='td-post-author-name')
            author_element = author_container.find('a') if author_container else None
            author = author_element.text.strip() if author_element else None

            date_posted_element = soup.find('time', class_='entry-date')
            date_posted = date_posted_element['datetime'].strip() if date_posted_element else None

            views_element = soup.find('div', class_='td-post-views')
            views = views_element.find('span', class_='td-nr-views-19100').text.strip() if views_element and views_element.find('span', class_='td-nr-views-19100') else None

            # Check if the element is found before accessing its properties
            image_url_element = soup.find('div', class_='td-post-featured-image')
            image_url = image_url_element.find('img')['src'] if image_url_element and image_url_element.find('img') else None

            # Ajouter les détails à la liste
            job_data.append({
                'Job Title': job_title,
                'Author': author,
                'Date Posted': date_posted,
                'Views': views,
                'Image URL': image_url,
                'URL': url  # Ajoutez l'URL de la page à la liste
            })
        except Exception as e:
            print(f"Une erreur s'est produite pour l'URL {url}: {str(e)}")
            job_data.append({
                'Job Title': "",
                'Author': "",
                'Date Posted': "",
                'Views': "",
                'Image URL': "",
                'URL': url  # Ajoutez l'URL de la page à la liste
            })

    # Créer un DataFrame avec les données extraites
    df = pd.DataFrame(job_data)
    alerte_emploi_df = pd.merge(df, df_alerteemploi, on='URL')
    # Afficher le DataFrame
    return alerte_emploi_df

# Appeler la fonction alerte_emploi pour obtenir le DataFrame combiné
alerte_emploi_df = alerte_emploi()

# Afficher le DataFrame combiné
alerte_emploi_df

# Emploi_jeune.ci

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

def agence_emploi_jeunes():
    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Liste des URLs à scraper
    urls = ["https://agenceemploijeunes.ci/site/offres-emplois?page={}".format(category) for category in range(31)]

    # Listes pour stocker les données
    job_titles = []
    publication_dates = []
    application_deadlines = []
    locations = []
    job_descriptions = []
    job_types = []
    diploma_requirements = []
    url_lien = []

    # Loop à travers chaque URL
    for url in urls:
        # Envoyer une requête au site web
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        # Trouver les annonces d'emploi
        job_listings = soup.find_all('div', class_='post-bx')

        # Extract data from each job listing
        for job_listing in job_listings:
            url_lien.append(url)

            # Titre du poste
            job_title = job_listing.find('h4').text.strip()
            job_titles.append(job_title)

            # Dates de publication et de candidature
            date_info = job_listing.find_all('li', {'class': ''})
            if date_info:
                publication_date = date_info[0].text.replace('Publié le:', '').strip()
                application_deadline = date_info[1].text.replace('Date limite:', '').strip()
                publication_dates.append(publication_date)
                application_deadlines.append(application_deadline)

            # Localisation
            location = date_info[2].text.replace('ABENGOUROU', '').strip()
            locations.append(location)

            # Description du poste
            job_description = job_listing.find('p').text.strip()
            job_descriptions.append(job_description)

            # Type de poste
            job_type = job_listing.find('span', {'class': 'pull-right'}).text.strip()
            job_types.append(job_type)

            # Exigences en diplôme
            diploma_requirement = job_listing.find('div', {'class': 'salary-bx'}).text.strip()
            diploma_requirements.append(diploma_requirement)

    # Créer un DataFrame Pandas
    data = {
        'Job Title': job_titles,
        'Publication Date': publication_dates,
        'Application Deadline': application_deadlines,
        'Location': locations,
        'Job Description': job_descriptions,
        'Job Type': job_types,
        'Diploma Requirement': diploma_requirements,
        "URL": url_lien
    }

    df_agenceemploijeunes = pd.DataFrame(data)

    # Utiliser la méthode str.extract pour extraire la valeur après "Diplôme :"
    df_agenceemploijeunes['Diplome'] = df_agenceemploijeunes['Diploma Requirement'].str.extract(r'Diplôme :[ \t]([^\n\r])')

    # Scrapper les détails supplémentaires à partir des URL
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # List to store job details
    all_job_details = []

    # Parcourir les liens
    for url in urls:
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)

        # Trouver toutes les offres d'emploi sur la page
        offer_links = soup.select('.post-bx h4 a')

        # Parcourir les liens d'offres
        for offer_link in offer_links:
            # Extraire l'URL de l'offre
            offer_url = offer_link.get('href')

            # Ajouter les détails à la liste
            all_job_details.append({'Offre_Link': offer_url, 'URL': url})

    # Fermer le navigateur à la fin
    driver.quit()

    # Créer un DataFrame avec les détails des offres d'emploi
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    df_agenceemploi_jeunes = pd.merge(df_agenceemploijeunes, all_job_details_df, on='URL')

    # Création d'un dictionnaire pour stocker les données
    job_data = {
        'Job Title': [],
        'Location': [],
        'Reference': [],
        'Number of Positions': [],
        'Closing Date': [],
        'Diploma': [],
        'Job Type': [],
        'Experience': [],
        'Education Level': [],
        'Gender': [],
        'Job Description': [],
        'Offre_Link': []
    }

    # Loop through each URL
    for url in df_agenceemploi_jeunes["Offre_Link"]:
        offre_url = url  # Sauvegarder l'URL même en cas d'exception
        try:
            # Send a request to the website
            req = requests.get(url, headers=headers)
            req.raise_for_status()  # Raise an error for unsuccessful responses
            soup = BeautifulSoup(req.text, 'html.parser')

            # Extract job details
            job_details = soup.find('div', class_='widget_getintuch')

            if job_details:
                # Extract data from job details
                ul_element = job_details.find('ul')
                if ul_element:
                    details_list = ul_element.find_all('li')

                    # Initialize variables to store details
                    location = reference = num_positions = closing_date = diploma = job_type = experience = education_level = gender = None

                    # Iterate through details
                    for detail in details_list:
                        label = detail.find('strong')
                        value_span = detail.find('span', class_='text-black-light')

                        if label and value_span:
                            label_text = label.text.strip()
                            value_text = value_span.text.strip()

                            if 'Lieu de travail' in label_text:
                                location = value_text
                            elif 'Reference' in label_text:
                                reference = value_text
                            elif 'Nombre de poste' in label_text:
                                num_positions = value_text
                            elif 'Date de clôture' in label_text:
                                closing_date = value_text
                            elif 'Diplôme' in label_text:
                                diploma = value_text
                            elif 'Type de contrat' in label_text:
                                job_type = value_text
                            elif 'Expérience professionnelle' in label_text:
                                experience = value_text
                            elif 'Niveau d\'études' in label_text:
                                education_level = value_text
                            elif 'Sexe' in label_text:
                                gender = value_text

                    # Append extracted details to the dictionary
                    job_data['Location'].append(location)
                    job_data['Reference'].append(reference)
                    job_data['Number of Positions'].append(num_positions)
                    job_data['Closing Date'].append(closing_date)
                    job_data['Diploma'].append(diploma)
                    job_data['Job Type'].append(job_type)
                    job_data['Experience'].append(experience)
                    job_data['Education Level'].append(education_level)
                    job_data['Gender'].append(gender)

                # Extract job title and description
                job_title_element = soup.find('h3', {'class': 'title-head'})
                if job_title_element:
                    job_title = job_title_element.text.strip()
                    job_data['Job Title'].append(job_title)

                    job_description_info = soup.find('div', {'class': 'job-info-box'}).find('ul')
                    if job_description_info:
                        job_description_text = '\n'.join([li.text.strip() for li in job_description_info.find_all('li')])
                        job_data['Job Description'].append(job_description_text)
                    else:
                        job_data['Job Description'].append(None)
                else:
                    job_data['Job Title'].append(None)
                    job_data['Job Description'].append(None)
            else:
                job_data['Job Title'].append(None)
                job_data['Job Description'].append(None)

            # Append URL to the dictionary
            job_data['Offre_Link'].append(offre_url)

        except requests.exceptions.RequestException as e:
            print(f"An error occurred while accessing URL: {url}")
            print(e)
            # Ajouter l'URL même en cas d'exception
            job_data['Location'].append(None)
            job_data['Reference'].append(None)
            job_data['Number of Positions'].append(None)
            job_data['Closing Date'].append(None)
            job_data['Diploma'].append(None)
            job_data['Job Type'].append(None)
            job_data['Experience'].append(None)
            job_data['Education Level'].append(None)
            job_data['Gender'].append(None)
            job_data['Job Title'].append(None)
            job_data['Job Description'].append(None)
            job_data['Offre_Link'].append(offre_url)

    # Create DataFrame from the collected data
    df_jobs = pd.DataFrame(job_data)
    agenceemploi_jeunes_df = pd.merge(df_agenceemploi_jeunes, df_jobs, on='Offre_Link')
    equivalences = {
    "Job Title_x": "INTITULE_DU_POSTE",
    "Publication Date": "DATE_DE_PUBLICATION",
    "Application Deadline": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Location_x": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Job Description_x": "DESCRIPTION_DU_POSTE",
    "Job Type_x": "TYPE_DE_CONTRAT_DU_POSTE",
    "Diploma Requirement": None,
    "URL": None,
    "Diplome": "DIPLOME_REQUIS",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    # Colonnes sans équivalence
    "Location_y": None,
    "Reference": None,
    "Number of Positions": None,
    "Closing Date": None,
    "Diploma": None,
    "Job Type_y": None,
    "Experience": None,
    "Education Level": None,
    "Gender": None,
    "Job Description_y": None}
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    agenceemploi_jeunes_df = renommer_colonnes(agenceemploi_jeunes_df, equivalences)
    
    return agenceemploi_jeunes_df
agence_emploi_jeunes()

,INTITULE_DU_POSTE,DATE_DE_PUBLICATION,DATE_D_EXPIRATION_DE_L_OFFRE,LIEU_DU_POSTE_DE_TRAVAIL,DESCRIPTION_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,Diploma Requirement,URL,DIPLOME_REQUIS,SITE_WEB_DE_L_ENTREPRISE,...,Location_y,Reference,Number of Positions,Closing Date,Diploma,Job Type_y,Experience,Education Level,Gender,Job Description_y
0,STAGIAIRE ASSISTANT DE DIRECTION,29 02 2024,09 03 2024,SEYDOUGOU,-ACCUEIL DES VISITEURS\r\n\r\n-PRISE DE RENDEZ...,STAGE DE VALIDATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,SEYDOUGOU,ODIE-33985-02-2024,4,09/03/2024,BTS,STAGE DE VALIDATION,,BAC+2,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...
1,STAGIAIRE ASSISTANT DE DIRECTION,29 02 2024,09 03 2024,SEYDOUGOU,-ACCUEIL DES VISITEURS\r\n\r\n-PRISE DE RENDEZ...,STAGE DE VALIDATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,SEYDOUGOU,ODIE-33985-02-2024,4,09/03/2024,BTS,STAGE DE VALIDATION,,BAC+2,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...
2,STAGIAIRE ASSISTANT DE DIRECTION,29 02 2024,09 03 2024,SEYDOUGOU,-ACCUEIL DES VISITEURS\r\n\r\n-PRISE DE RENDEZ...,STAGE DE VALIDATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,SEYDOUGOU,ODIE-33985-02-2024,4,09/03/2024,BTS,STAGE DE VALIDATION,,BAC+2,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...
3,STAGIAIRE ASSISTANT DE DIRECTION,29 02 2024,09 03 2024,SEYDOUGOU,-ACCUEIL DES VISITEURS\r\n\r\n-PRISE DE RENDEZ...,STAGE DE VALIDATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,SEYDOUGOU,ODIE-33985-02-2024,4,09/03/2024,BTS,STAGE DE VALIDATION,,BAC+2,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...
4,STAGIAIRE ASSISTANT DE DIRECTION,29 02 2024,09 03 2024,SEYDOUGOU,-ACCUEIL DES VISITEURS\r\n\r\n-PRISE DE RENDEZ...,STAGE DE VALIDATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,SEYDOUGOU,ODIE-33985-02-2024,4,09/03/2024,BTS,STAGE DE VALIDATION,,BAC+2,MASCULIN,STAGE DE VALIDATION\nDate de clôture:\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,EMPLOYE POLYVALENT STAGIAIRE,04 12 2023,29 02 2024,GUIGLO,EFFECTUER LA MISE EN RAYON\r\n\r\nLE CONDITION...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,GUIGLO,GUIG-31701-11-2023,1,29/02/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
32996,EMPLOYE POLYVALENT STAGIAIRE,04 12 2023,29 02 2024,GUIGLO,EFFECTUER LA MISE EN RAYON\r\n\r\nLE CONDITION...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,GUIGLO,GUIG-31701-11-2023,1,29/02/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
32997,EMPLOYE POLYVALENT STAGIAIRE,04 12 2023,29 02 2024,GUIGLO,EFFECTUER LA MISE EN RAYON\r\n\r\nLE CONDITION...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,GUIGLO,GUIG-31701-11-2023,1,29/02/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
32998,EMPLOYE POLYVALENT STAGIAIRE,04 12 2023,29 02 2024,GUIGLO,EFFECTUER LA MISE EN RAYON\r\n\r\nLE CONDITION...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,GUIGLO,GUIG-31701-11-2023,1,29/02/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...


# rmo-jobcenter.com

In [10]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

def rmo_jobcenter():
    # Liste des URLs des pages d'offres d'emploi
    urls = [
        "https://rmo-jobcenter.com/fr/nos-offres-emploi.html",
        # Ajoutez d'autres URLs au besoin
    ]

    # Configurez Selenium pour s'exécuter en mode headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    # Liste pour stocker les données des offres d'emploi
    all_job_data = []

    # Base URL of the website
    base_url = "https://rmo-jobcenter.com"

    # Boucle à travers chaque URL
    for url in urls:
        # Initialisez le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Chargez la page avec Selenium
        driver.get(url)

        # Récupérez le code source de la page après l'exécution du JavaScript
        page_source = driver.page_source

        # Fermez le pilote Selenium
        driver.quit()

        # Utilisez BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Trouvez la table contenant les offres d'emploi
        table = soup.find('table', class_='liste')

        # Liste pour stocker les données des offres d'emploi pour une URL spécifique
        job_data = []

        # Boucle à travers chaque ligne de la table (sauf la première qui contient les en-têtes)
        for row in table.find_all('tr')[1:]:
            # Extrayez les données de chaque colonne
            columns = row.find_all('td')
            date = columns[0].text.strip()
            filiale = columns[1].text.strip()
            fonction = columns[2].text.strip()
            secteur = columns[3].text.strip()
            reference = columns[4].text.strip()
            details_url = columns[5].find('a')['href']

            # Rendez l'URL absolue en la combinant avec l'URL de base.
            absolute_url = f"{base_url}/{details_url}"

            # Stockez les données dans un dictionnaire
            job_entry = {
                'Date': date,
                'Filiale': filiale,
                'Fonction': fonction,
                'Secteur': secteur,
                'Référence / Statut': reference,
                'Détails URL': absolute_url,
                "URL": url
            }

            job_data.append(job_entry)

        # Ajoutez les données de cette URL à la liste globale
        all_job_data.extend(job_data)

    # Créez un DataFrame avec toutes les données extraites
    df_jobcenter = pd.DataFrame(all_job_data)

    # Liste des URLs des pages d'offres d'emploi détaillées
    detail_urls = list(df_jobcenter['Détails URL'])

    # Liste pour stocker les données détaillées
    all_detail_data = []

    # Boucle à travers chaque URL détaillée
    for detail_url in detail_urls:
        # Initialisez le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Chargez la page avec Selenium
        driver.get(detail_url)

        # Récupérez le code source de la page après l'exécution du JavaScript
        page_source = driver.page_source

        # Fermez le pilote Selenium
        driver.quit()

        # Utilisez BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Trouvez la div contenant les informations détaillées
        details_div = soup.find('div', {'id': 'content_articles'})

        # Extract details from the div
        job_title = details_div.find('div', {'id': 'h2_imprime'}).text.strip()
        job_description = details_div.find('div', {'class': 'text-content'}).text.strip()

        # Stockez les données dans un dictionnaire
        detail_entry = {
            'Job Title': job_title,
            'Job Description': job_description,
            'Job URL': detail_url
            # Add more details as needed
        }

        # Ajoutez les données de cette URL à la liste globale
        all_detail_data.append(detail_entry)

    # Créez un DataFrame avec toutes les données extraites
    df_details = pd.DataFrame(all_detail_data)
    df_details[['Poste', 'Niveau', 'Sous Poste']] = df_details['Job Title'].str.split(' - ', expand=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_jobcenter['Job Title'] = df_details['Job Title']
    df_jobcenter['Job Description'] = df_details['Job Description']
    df_jobcenter['Job URL'] = df_details['Job URL']
    df_jobcenter['Poste'] = list(df_details['Poste'])
    df_jobcenter['Niveau'] = list(df_details['Niveau'])
    df_jobcenter['Sous Poste'] =  list(df_details['Sous Poste'])
    equivalences = {
    "Date": "DATE",
    "Filiale": "FILIALE",
    "Fonction": "FONCTION",
    "Secteur": "SECTEUR",
    "Référence / Statut": "REFERENCE_STATUT",
    "Détails URL": "DETAILS_URL",
    "Job Title": "INTITULE_DU_POSTE",
    "Job Description": "DESCRIPTION_DU_POSTE",
    "Job URL": "URL_DU_POSTE",
    "Poste": "POSTE",
    "Niveau": "NIVEAU",
    "Sous Poste": "SOUS_POSTE"}
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    df_jobcenter = renommer_colonnes(df_jobcenter, equivalences)
    # Réorganiser les colonnes selon vos besoins

    return df_jobcenter

# Appeler la fonction rmo_jobcenter pour obtenir le DataFrame combiné
rmo_jobcenter_df = rmo_jobcenter()

# Afficher le DataFrame combiné
rmo_jobcenter_df

,DATE,FILIALE,FONCTION,SECTEUR,REFERENCE_STATUT,DETAILS_URL,URL,INTITULE_DU_POSTE,DESCRIPTION_DU_POSTE,URL_DU_POSTE,POSTE,NIVEAU,SOUS_POSTE
0,29/02/2024,,Directeur des Opérations (H/F),Ressources Humaines,Réf: #396900Réception candidatureexpire le 15/...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Directeur des Opérations (H/F) - Autre - Récep...,Le PosteRMO\nMali recrute pour un\nbureau d’ét...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Directeur des Opérations (H/F),Autre,Réception candidature - expire le 15/03/2024
1,26/02/2024,,Receptionniste (H/F),Immobilier,Réf: #489229Réception candidatureexpire le 26/...,https://rmo-jobcenter.com/fr/senegal/offres-em...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Receptionniste (H/F) - CDI - Réception candida...,Le PosteNous recherchons un Réceptionniste H/F...,https://rmo-jobcenter.com/fr/senegal/offres-em...,Receptionniste (H/F),CDI,Réception candidature - expire le 26/03/2024
2,22/02/2024,,Chef d’équipe recrutement Senior (H/F),Ressources Humaines,Réf: #995393Réception candidatureexpire le 22/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chef d’équipe recrutement Senior (H/F) - CDI -...,"Le Poste\nRMO\nImplanté en Afrique,\nRMO vous ...",https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Chef d’équipe recrutement Senior (H/F),CDI,Réception candidature - expire le 22/03/2024
3,21/02/2024,,Contrôleur de Gestion Groupe (H/F) (H/F),Ressources Humaines,Réf: #583254Réception candidatureexpire le 13/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Contrôleur de Gestion Groupe (H/F) (H/F) - CDI...,Le PosteEn étroite collaboration avec la Direc...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Contrôleur de Gestion Groupe (H/F) (H/F),CDI,Réception candidature - expire le 13/03/2024
4,21/02/2024,,Chauffeur (H/F),Organisation Internationale / Association / ONG,Réf: #322470Réception candidatureexpire le 06/...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chauffeur (H/F) - Autre - Réception candidatur...,Le PosteRMO\nMali recrute pour\nune Ambassade ...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Chauffeur (H/F),Autre,Réception candidature - expire le 06/03/2024
5,20/02/2024,,Coordinateur Bilingue Atelier de réparation (H/F),Commercial / Vente/ Distribution,Réf: #282400Réception candidatureexpire le 20/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Coordinateur Bilingue Atelier de réparation (H...,Le PosteDans l’optique de continuer \nd’assure...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Coordinateur Bilingue Atelier de réparation (H/F),CDI,Réception candidature - expire le 20/03/2024
6,20/02/2024,,Responsable Ressources Humaines (H/F),Organisation Internationale / Association / ONG,Réf: #435655Réception candidatureexpire le 01/...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Responsable Ressources Humaines (H/F) - Autre ...,Le PosteRMO MALI\nRMO Mali recrute\nun Respons...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Responsable Ressources Humaines (H/F),Autre,Réception candidature - expire le 01/03/2024
7,19/02/2024,,Chef de rayon (H/F),Commercial / Vente/ Distribution,Réf: #516522Réception candidatureexpire le 30/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chef de rayon (H/F) - CDI - Réception candidat...,"Le PosteRMOImplanté en Afrique, RMO vous propo...",https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Chef de rayon (H/F),CDI,Réception candidature - expire le 30/04/2024
8,19/02/2024,,Auditeur Interne (H/F),BTP / Architecture,Réf: #257136Réception candidatureexpire le 20/...,https://rmo-jobcenter.com/fr/senegal/offres-em...,https://rmo-jobcenter.com/fr/nos-offres-emplo

# projobivoire

In [11]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def projobivoire():
    def extract_text(element, tag_name=None):
        if element and tag_name:
            tag = element.find(tag_name)
            return tag.text.strip() if tag else ""
        return ""

    def clean_text(text):
        return text.replace('\r\n', '').replace('\xa0', '')

    def scrape_projobivoire_page(page_url):
        job_data_list = []

        for url in page_url:
            try:
                response = requests.get(url, timeout=500)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                    print(f"Erreur de connexion à {url} : {e}")
                    continue

            soup = BeautifulSoup(response.text, 'html.parser')

            job_items = soup.find_all('div', class_='loop-item-wrap list')

            if not job_items:
                print(f"Aucun élément de travail trouvé pour l'URL : {url}")
                continue

            for job_item in job_items:
                title_tag = job_item.find('h3', class_='loop-item-title')
                title = extract_text(title_tag, 'a')

                job_type_tag = job_item.find('span', class_='job-type')
                job_type = extract_text(job_type_tag, 'span')

                job_date_posted = soup.find('span', class_='job-date__posted').text.strip()

                job_date_closing_tag = soup.find('span', class_='job-date__closing')
                job_date_closing = job_date_closing_tag.text.strip() if job_date_closing_tag else ""

                job_date_closing = job_date_closing.lstrip('-').strip()

                category_tag = job_item.find('span', class_='job-category')
                category = extract_text(category_tag, 'a')

                # Ajout de ces lignes pour extraire l'URL de l'e-mail
                email_url_tag = job_item.find('span', class_='noo-tool-email-job')
                email_url = email_url_tag['data-url'] if email_url_tag else ""

                data = {
                    'Title': title,
                    'Type': job_type,
                    'DatePosted': job_date_posted,
                    'DateClosing': job_date_closing,
                    'Category': category,
                    'EmailURL': email_url,
                    'URL': url
                }

                job_data_list.append(data)

        return job_data_list

    # Liste des URL de pages avec plusieurs offres d'emploi
    page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(546)]

    # Scrape des détails de chaque offre d'emploi sur les pages
    job_data_list = scrape_projobivoire_page(page_urls)

    # Création d'un DataFrame à partir de la liste des données d'emploi
    df_projobivoire = pd.DataFrame(job_data_list)

    # Affichage du DataFrame

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Title": "INTITULE_DU_POSTE",
        "Type": "TYPE_DE_CONTRAT_DU_POSTE",
        "DatePosted": "DATE_DE_PUBLICATION",
        "DateClosing": "DATE_D_EXPIRATION_DE_L_OFFRE",
        "Category": "CATEGORIE",
        "EmailURL": None,
        "URL": "SITE_WEB_DE_L_ENTREPRISE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_projobivoire = renommer_colonnes(df_projobivoire, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_projobivoire

# Appeler la fonction projobivoire pour obtenir le DataFrame des offres d'emploi
df_projobivoire = projobivoire()

# Afficher le DataFrame
df_projobivoire

,INTITULE_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,DATE_DE_PUBLICATION,DATE_D_EXPIRATION_DE_L_OFFRE,CATEGORIE,EmailURL,SITE_WEB_DE_L_ENTREPRISE
0,JURISTE | SECRETAIRE | COMMERCIAUX A OBROU SER...,Stage,1 mars 2024,8 mars 2024,Commerce/Ventes,https://projobivoire.com/jobs/juriste-secretai...,https://projobivoire.com/page/0/
1,CHARGES D’ETUDES CVC ET PLOMBERIE,Emploi,1 mars 2024,8 mars 2024,Electromécanique,https://projobivoire.com/jobs/charges-detudes-...,https://projobivoire.com/page/0/
2,La PALMCI recrute en ce moment – 39 Postes dis...,Emploi,1 mars 2024,8 mars 2024,Agriculture,https://projobivoire.com/jobs/la-palmci-recrut...,https://projobivoire.com/page/0/
3,La PALMCI recrute 06 Superviseurs – CDI,Emploi,1 mars 2024,8 mars 2024,Agriculture,https://projobivoire.com/jobs/la-palmci-recrut...,https://projobivoire.com/page/0/
4,La PALMCI recrute 02 Chefs de Zone,Emploi,1 mars 2024,8 mars 2024,Agronomie,https://projobivoire.com/jobs/la-palmci-recrut...,https://projobivoire.com/page/0/
...,...,...,...,...,...,...,...
9277,SAGES-CI recrute CHEF DE PROJET COMMERCIAL,Emploi,28 juin 2021,1 juillet 2021,Commerce/Ventes,https://projobivoire.com/jobs/sages-ci-recrute...,https://projobivoire.com/page/545/
9278,TANASA LUBRIFIANTS recrute ASSISTANT CHEF DE P...,Emploi,28 juin 2021,1 juillet 2021,Assistanat de Direction,https://projobivoire.com/jobs/tanasa-lubrifian...,https://projobivoire.com/page/545/
9279,RAZEL-BEC CÔTE D’IVOIRE recrute CHEF LABORATOI...,Emploi,28 juin 2021,1 juillet 2021,Génie Civil/Travaux publics,https://projobivoire.com/jobs/razel-bec-cote-d...,https://projobivoire.com/page/545/
9280,PSI-CI recrute DEUX (02) COMPTABLES,Emploi,28 juin 2021,1 juillet 2021,Finances/Comptabilité,https://projobivoire.com/jobs/psi-ci-recrute-d...,https://projobivoire.com/page/545/


# ci.talent

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def talent_ci():
    def extract_text(element, tag_name=None):
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""

    def clean_text(text):
        return text.replace('\r\n', '').replace('\xa0', '')

    def scrape_talent_com(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_wrappers = soup.find_all('div', class_='card card__job')

        data_list = []

        for wrapper in job_wrappers:
            title_tag = wrapper.find('h2', class_='card__job-title')
            title = extract_text(title_tag, 'a')
            
            employer_location_tag = wrapper.find('div', class_='card__job-empnameLocation')
            #employer = extract_text(employer_location_tag.find('div', class_='card__job-location'))  # Extract location from the inner div
            location= extract_text(employer_location_tag, 'div')

            employer_location_tag = wrapper.find('div', class_='card__job-empname-label')
            
            # Extracting employer and description from the div
            employer = employer_location_tag.text.strip() if employer_location_tag else None  # Extract location from the inner div
            
            description_tag = wrapper.find('div', class_='card__job-snippet-logo')
            description = clean_text(extract_text(description_tag, 'p'))

            data_list.append({
                'Title': title,
                'Location': location,
                'Employer': employer,
                'Description': description,
                'URL': url
            })

        df = pd.DataFrame(data_list)
        return df

    # List of URLs for talent.com jobs
    urls = [
         "https://ci.talent.com/jobs?l=Abidjan%2C+Abidjan&radius=15&p={}&k=&context=serp_pagination".format(category) for category in range(8)
        # Add more URLs as needed "https://ci.talent.com/jobs",
    ]

    # Initialize an empty DataFrame to store the results
    ci_talent = pd.DataFrame()

    # Scrape job information for each URL and concatenate the results
    for url in urls:
        df = scrape_talent_com(url)
        ci_talent = pd.concat([ci_talent, df], ignore_index=True)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Title": "INTITULE_DU_POSTE",
        "Location": "LIEU_DU_POSTE_DE_TRAVAIL",
        "Employer": "RAISON_SOCIALE_DE_L_ENTREPRISE",
        "Description": "DESCRIPTION_DU_POSTE",
        "URL": "SITE_WEB_DE_L_ENTREPRISE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    ci_talent = renommer_colonnes(ci_talent, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return ci_talent

# Appeler la fonction talent_ci pour obtenir le DataFrame des offres d'emploi
df_talent_ci = talent_ci()

# Afficher le DataFrame
df_talent_ci

,INTITULE_DU_POSTE,LIEU_DU_POSTE_DE_TRAVAIL,RAISON_SOCIALE_DE_L_ENTREPRISE,DESCRIPTION_DU_POSTE,SITE_WEB_DE_L_ENTREPRISE
0,Country Technical Sales Manager – West Africa ...,"Abidjan, Abidjan",Claire Joster,PresentaciónClaire Joster is a headhunting fir...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
1,Conseiller national pour la mise à l'échelle (...,"Abidjan, Abidjan",Options Consultancy Services Ltd,Nous sommes une équipe mondiale d'experts et d...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
2,Training Specialist,"Abidjan, Abidjan",CARGILL,Entreprise CARGILL Site Internet. Secteur d ac...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
3,"Monitoring, Evaluation and Learning (MEL) Advisor","Abidjan, Abidjan",ENGENDERHEALTH,Entreprise ENGENDERHEALTH Site Internet. Secte...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
4,UX / UI Designer (M/F),"Abidjan, Abidjan",AFRICASHORE,Entreprise AFRICASHORE Site Internet. Secteur ...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
...,...,...,...,...,...
203,Operations manager - Abidjan - Pasta,"Abidjan, Abidjan",Michael Page,Reporting to the General Manager & Regional Vi...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
204,External Relations Officer,"Abidjan, Abidjan Autonomous District",WHO,DESCRIPTION OF DUTIES1.elaborer et mettre en œ...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
205,Intern - Business Development (For Current Stu...,"Abidjan, Abidjan",VISA,Job Description And ResponsibilitiesEveryone a...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
206,RECRUTEMENT AUDITEURS INTERNES INFORMATIQUE SE...,"Abidjan, Abidjan",PwC,Description & SummaryManagement Level. Senior ...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...


# worldbankgroup mondiale

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def mondiale_ci():
    # Fonction pour extraire les données d'une page
    def scrape_page(url):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Lèvera une exception si la requête a échoué
            soup = BeautifulSoup(response.content, "html.parser")
            job_rows = soup.find("table", class_="results-table").find("tbody").find_all("tr")

            # Listes pour stocker les données de cette page
            job_titles = []
            locations = []
            job_families = []
            deadlines = []

            # Parcourir chaque ligne du tableau et extraire les informations nécessaires
            for row in job_rows:
                # Récupérer le titre de l'emploi
                job_title = row.find("a").text.strip()
                job_titles.append(job_title)

                # Récupérer l'emplacement
                location = row.find_all("td")[1].text.strip()
                locations.append(location)

                # Récupérer la famille d'emploi
                job_family = row.find_all("td")[2].text.strip()
                job_families.append(job_family)

                # Récupérer la date limite d'application
                deadline = row.find_all("td")[3].text.strip()
                deadlines.append(deadline)

            # Retourner les données de cette page sous forme de DataFrame
            data = {
                "Job Title": job_titles,
                "Location": locations,
                "Job Family": job_families,
                "Deadline": deadlines
            }
            return pd.DataFrame(data)
        except requests.exceptions.RequestException as e:
            print("Une erreur s'est produite lors de la requête:", e)
            return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur

    # URL de la première page
    base_url = "https://worldbankgroup.csod.com/ats/careersite/search.aspx?site=1&c=worldbankgroup&sid=%5e%5e%5eFLGscZMYY2RrwVaMR%2ftHYw%3d%3d"

    # Créer une liste pour stocker les DataFrames de chaque page
    dfs = []

    # Extraire les données de la première page
    dfs.append(scrape_page(base_url))

    # Trouver le nombre total de pages
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")
    pagination_span = soup.find("span", class_=re.compile(r"\btext\b"), text=re.compile(r"\d+"))
    if pagination_span:
        num_pages = int(pagination_span.text.strip())
    else:
        num_pages = 1

    # Boucle à travers chaque page et extraire les données
    for page_num in range(1, num_pages + 1):
        page_url = f"{base_url}&pg={page_num}"
        df = scrape_page(page_url)
        if not df.empty:  # Vérifier si le DataFrame n'est pas vide
            dfs.append(df)

    # Concaténer tous les DataFrames en un seul
    df_mondiale = pd.concat(dfs, ignore_index=True)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Job Title": "INTITULE_DU_POSTE",
        "Location": "LIEU_DU_POSTE_DE_TRAVAIL",
        "Job Family": "SECTEUR",
        "Deadline": "DATE_D_EXPIRATION_DE_L_OFFRE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_mondiale = renommer_colonnes(df_mondiale, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_mondiale

# Appel de la fonction principale

mondiale_ci()

,INTITULE_DU_POSTE,LIEU_DU_POSTE_DE_TRAVAIL,SECTEUR,DATE_D_EXPIRATION_DE_L_OFFRE
0,"​​ET Temporary, Program Assistant, GPE​","Paris,France",Operations,3/12/2024
1,(Senior) Environmental and Social Advisory Spe...,"Washington, DC,United States",Advisory Services,3/7/2024
2,Anti-Harassment Counsel,"New Delhi,India",Ethics & Internal Justice Systems,3/18/2024
3,Associate Financial Officer - Cash Management,"Washington, DC,United States",Financial Sector,3/15/2024
4,Chief Evaluation Officer,"Washington, DC,United States",Environment,3/5/2024
5,Country Officer,"Addis Ababa,Ethiopia",Operations,3/7/2024
6,Data Management Specialist,"Washington, DC,United States",Economics,3/25/2024
7,Deputy Chief Economist,"Washington, DC,United States",Economics,3/18/2024
8,E T Consultant,"Washington, DC,United States",Economics,3/1/2024
9,E T Consultant,"Washington, DC,United States",Economics,3/1/2024


# yop_l_frii

In [14]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def yop_l_frii():
    # List of URLs
    urls = [
        f"https://yop.l-frii.com/offres-demplois/{category}/" for category in range(3276)
        # Add more URLs as needed
    ]

    job_data = []

    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
            soup = BeautifulSoup(response.content, 'html.parser')

            job_articles = soup.find_all('article', class_='type-emploi')

            for article in job_articles:
                job_title_element = article.find('h2', class_='elementor-heading-title')
                job_title = job_title_element.text.strip() if job_title_element else None

                job_link = article.find('a', href=True)['href']

                job_image_element = article.find('img', class_='attachment-large')
                job_image = job_image_element['src'] if job_image_element else None

                job_data.append({
                    "Job Title": job_title,
                    "Job Link": job_link,
                    "Job Image": job_image,
                    "Source URL": url
                })
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while fetching data from {url}: {e}")

    df_yop_l_frii = pd.DataFrame(job_data)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Job Title": "INTITULE_DU_POSTE",
        "Job Link": "URL_DU_POSTE",
        "Job Image": None,
        "Source URL": None
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if
                              nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_yop_l_frii = renommer_colonnes(df_yop_l_frii, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_yop_l_frii

# Appel de la fonction principale
yop_l_frii()

,INTITULE_DU_POSTE,URL_DU_POSTE,Job Image,Source URL
0,Bibliothèques Sans Frontières recrute pour ce ...,https://yop.l-frii.com/emploi/bibliotheques-sa...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
1,L’ONG internationale SOLTHIS recrute pour ces ...,https://yop.l-frii.com/emploi/long-internation...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
2,UNOPS- le Bureau des Nations Unies pour les se...,https://yop.l-frii.com/emploi/unops-le-bureau-...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
3,La multinationale suisse NESTLÉ recrute pour c...,https://yop.l-frii.com/emploi/la-multinational...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
4,Le Bureau régional de l’UNICEF recrute pour ce...,https://yop.l-frii.com/emploi/le-bureau-region...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
...,...,...,...,...
39307,L’IRC recrute pour ce poste (03 Novembre 2021),https://yop.l-frii.com/emploi/lirc-recrute-pou...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/
39308,L’OIM recrute pour ce poste (30 Octobre 2021),https://yop.l-frii.com/emploi/loim-recrute-pou...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/
39309,La Banque Mondiale recrute pour ce poste (03 N...,https://yop.l-frii.com/emploi/la-banque-mondia...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/
39310,La Banque africaine de développement recrute p...,https://yop.l-frii.com/emploi/la-banque-africa...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/


# emploi_ci

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

from selenium import webdriver

def emploi_ci():
    def extract_text(element, tag_name=None):
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""

    def clean_text(text):
        return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

        data_list = []

        for wrapper in job_description_wrappers:
            h5_tag = wrapper.find('h5')
            poste = extract_text(h5_tag, 'a')

            job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
            date_and_company = job_recruiter_tag.text.strip().split('|')
            date = date_and_company[0].strip() if date_and_company else ""
            entreprise = extract_text(job_recruiter_tag, 'a')

            description_tag = wrapper.find('div', class_='search-description')
            description = clean_text(description_tag.text.strip()) if description_tag else ""

            region_tag = wrapper.find('p', text='Région de :')
            region = extract_text(region_tag) if region_tag else ""

            data_list.append({
                'Poste': poste,
                'Entreprise': entreprise,
                'Date': date,
                'Description': description,
                'Région': region,
                'URL' : url
            })

        df = pd.DataFrame(data_list)
        return df

    # Liste des liens
    categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
    #categories=["31"]
    # Liste d'URLs générées
    urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

    # Créer un DataFrame à partir des liens
    df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)


    from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

    # Liste des liens

    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in list(df["URL"]):
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_="job-description-wrapper")

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h5')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link, 'URL' :url})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        # Afficher le DataFrame
        #print(all_job_details_df)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    #emploi_df = pd.merge(df, all_job_details_df, on='URL')
    df["URL"]=list(all_job_details_df["URL"])
    df["Offre_Link"]=list(all_job_details_df["Offre_Link"])

    # Fonction pour extraire les informations d'une page
    def extract_information(url):
        try:
            response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
            response.raise_for_status()
            response.encoding = 'utf-8'

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraction des informations sur l'entreprise
            company_info = soup.select_one('.job-ad-company')
            entreprise = {
                "Offre_Link" : url,
                'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
                'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
                'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
            }

            # Extraction des informations sur l'annonce
            annonce_info = soup.select_one('.job-ad-details')
            annonce = {
                'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
                'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
                'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
                'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
                'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
                'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
                'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
                'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
                'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
                'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
                'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
                'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
            }

            return {'entreprise': entreprise, 'annonce': annonce}

        except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
            print(f"Erreur lors de la requête {url}: {e}")
            # Relancer la requête
            entreprise = {
                "Offre_Link" : url,
                'Nom': "",
                'Secteur d´activité': "",
                'Description de l\'entreprise':""}
            annonce = {'Poste':"",
                       'Missions': "",
                       'Profil recherché':"",
                       'Métier':"",
                       'Secteur d´activité (de l\'annonce)':"",
                       'Type de contrat':"",
                       'Région': "",
                       'Ville':"",
                       'Niveau d\'expérience':"",
                       'Niveau d\'études':"",
                       'Compétences clés':"",
                       'Nombre de poste(s)':""}


            return {'entreprise': entreprise, 'annonce': annonce}

    # Liste des URLs
    urls = list(emploi_df['Offre_Link'])

    # Initialisation d'une liste pour stocker les DataFrames
    df_list = []

    # Boucle à travers chaque URL
    for url in urls:
        data = extract_information(url)

        # Si la requête a échoué, passez à l'URL suivante
        if data is None:
            continue

        # Création du DataFrame pour chaque URL
        df = pd.DataFrame([data['entreprise'] | data['annonce']])

        # Ajout du DataFrame à la liste
        df_list.append(df)

    # Concaténation des DataFrames de chaque URL
    result_df = pd.concat(df_list, ignore_index=True)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    result_df['Intitulé du poste'] = list(df["Poste"])
    result_df['Entreprise'] = list(df["Entreprise"])
    result_df['Date'] = list(df["Date"])
    result_df['Description'] = list(df["Description"])
    result_df['Lieu du poste de travail'] = list(df["Région"])
    result_df['URL'] = list(df["URL"])
    result_df['Offre_Link'] = list(df["Offre_Link"])


# Réorganiser les colonnes selon vos besoins
#Poste 	Entreprise 	Date 	Description 	Région 	URL 	Offre_Link

    return result_df

# Appel de la fonction pour obtenir le DataFrame
emploi_df=emploi_ci()
emploi_df

Erreur de connexion à https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A31 : HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A31 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A8D8FA0AF0>, 'Connection to www.emploi.ci timed out. (connect timeout=500)'))
Erreur de connexion à https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A1127 : HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A1127 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001A8D8FA02E0>, 'Connection to www.emploi.ci timed out. (connect timeout=500)'))
Erreur de connexion à https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A29 : HTTPSConnectionPool(host='www.emploi.

KeyError: 'URL'